In [45]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


In [46]:
# Load the crypto_data.csv dataset.
file_path = "C:\\Users\\shail\\Cryptocurrencies\\Resources\\crypto_data.csv"

crypto_df= pd.read_csv(file_path,index_col = 0 )
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [47]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df['IsTrading']== True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [48]:
# Keep all the cryptocurrencies that have a working algorithm.
# YOUR CODE HERE
crypto_df = crypto_df[crypto_df['Algorithm'] != "NA"]


In [49]:
# Remove the "IsTrading" column. 
del crypto_df['IsTrading']
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [50]:
# Remove rows that have at least 1 null value.

crypto_df = crypto_df.dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [51]:
# Keep the rows where coins are mined.

crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [52]:
# Create a new DataFrame that holds only the cryptocurrencies names.

crypto_names = crypto_df['CoinName']
crypto_names.head()

42        42 Coin
404       404Coin
1337    EliteCoin
BTC       Bitcoin
ETH      Ethereum
Name: CoinName, dtype: object

In [53]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
#del crypto_df['CoinName']
crypto_df.drop(columns='CoinName',inplace = True)

crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [54]:
# Use get_dummies() to create variables for text features.

#Added totalCoinSupply also in get_dummies because its datatype was object and needed to convert it into int before scaling it
X = pd.get_dummies(crypto_df, columns =['Algorithm','ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# Standardize the data with StandardScaler().
data_scaler = StandardScaler()

#fit the scaler
X_scaled = data_scaler.fit_transform(X)
X_scaled

#X.dtypes

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

Deliverable 2: Reducing Data Dimensions Using PCA
# Deliverable 2: Reducing Data Dimensions Using PCA


In [56]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

In [57]:
# Create a DataFrame with the three principal components.

pcs_df = pd.DataFrame(data = X_pca, columns = ["principal component 1","principal component 2","principal component 3"], 
                        index = X.index)

pcs_df.head()

,principal component 1,principal component 2,principal component 3
42,-0.336997,1.025661,-0.390178
404,-0.320339,1.025758,-0.390369
1337,2.290050,1.635396,-0.578083
BTC,-0.153635,-1.304728,0.131041
ETH,-0.150367,-2.023175,0.270029


# Deliverable 3: Clustering Crytocurrencies Using K-Means
Finding the Best Value for k Using the Elbow Curve


In [58]:
# Create an elbow curve to find the best value for K.
# Use intertia to create an elbow curve 
inertia =[]
k = list(range(1,12))

print(k)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [59]:
# Initialize the K-Means model.
for i in k :
    km_model = KMeans(n_clusters=i, random_state=0)
    km_model.fit(pcs_df)
    inertia.append(km_model.inertia_)

elbow_data = {"k" :k, "inertia" : inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k",y="inertia", xticks=k,title="Elbow Curve")

#predictions = km_model.predict(pcs_df)


# Predict clusters

C:\Users\shail\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

In [60]:
#Here K value is 4

# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0,
       3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 0,
       0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 0, 3, 0, 0, 3, 0, 3, 0, 3,
       0, 3, 0, 0, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0,
       3, 0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 3, 0, 0,
       0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0,
       0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0, 3,
       0, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 0, 3, 0,

In [61]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.merge(crypto_df,pcs_df, left_index=True, right_index =True)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.merge(clustered_df,crypto_names, left_index=True, right_index=True)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.336997,1.025661,-0.390178,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.320339,1.025758,-0.390369,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.290050,1.635396,-0.578083,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.153635,-1.304728,0.131041,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.150367,-2.023175,0.270029,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.160983,-1.131023,0.023571,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.410041,1.261034,-0.383809,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.162098,-2.238295,0.227393,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.148810,-2.023279,0.270019,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.157695,-2.058334,0.281619,ZCash,3


# Deliverable 4: Visualizing Cryptocurrencies Results


3D-Scatter with Clusters

In [77]:
# Creating a 3D-Scatter with the PCA data and the clusters

#clustered_df.hvplot.scatter(x="sepal_length", y="sepal_width", by="class")


fig = px.scatter_3d(clustered_df, x="principal component 1", y="principal component 2", 
                    z="principal component 3", color="class", symbol="class",hover_name="CoinName",
                     hover_data=["Algorithm"],
                     width=800)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()


In [65]:
#Create a table with tradable cryptocurrencies

clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType','TotalCoinSupply','TotalCoinsMined','class'], 
                          sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [68]:
#Print the total number of tradable cryptocurrencies in the clustered_df DataFrame

clustered_df['CoinName'].unique()

clustered_df['CoinName'].count()

532

In [71]:
clustered_df.dtypes

Algorithm                 object
ProofType                 object
TotalCoinsMined          float64
TotalCoinSupply           object
principal component 1    float64
principal component 2    float64
principal component 3    float64
CoinName                  object
class                      int32
dtype: object

In [76]:
# Use the MinMaxScaler().fit_transform method to
# scale the TotalCoinSupply and TotalCoinSupply columns between the given range of zero and one

clustered_df["TotalCoinSupply"]= clustered_df["TotalCoinSupply"].astype(float)/100000000


scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])
scaled_data

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [99]:
#Create a new DataFrame using the clustered_df DataFrame index
#that contains the scaled data you created in Step 5

#clustered_df_scaled = pd.DataFrame(['scaled_data'])
#clustered_df_scaled.head()
#df = pd.DataFrame(data=numpy_data, index=["row1", "row2"], columns=["column1", "column2"])

clustered_df_scaled = pd.DataFrame(data=scaled_data, columns=["TotalCoinSupply", "TotalCoinsMined"],index=clustered_df.index)
clustered_df_scaled.head()


clustered_df_scaled['CoinName'] = clustered_df['CoinName']
clustered_df_scaled['class'] = clustered_df['class']


clustered_df_scaled.head()

,TotalCoinSupply,TotalCoinsMined,CoinName,class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3


In [100]:
#Create an hvplot scatter plot with x="TotalCoinsMined", y="TotalCoinSupply", and by="Class", 
#and have it show the CoinName when you hover over the the data point.


clustered_df_scaled.hvplot(
    kind="scatter",
    x= "TotalCoinsMined",
    y= "TotalCoinSupply",
    hover_cols=["CoinName"],
    by= "class")

:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)